# This example demonstrate how to train region classification (standart image classification model).

In [1]:
# Specify device
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 
# os.environ["CUDA_VISIBLE_DEVICES"] = ""  # For CPU inference
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"

In [2]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Nomeroff-Net path
NOMEROFF_NET_DIR = os.path.abspath('../')
sys.path.append(NOMEROFF_NET_DIR)

In [3]:
# auto download dataset
from NomeroffNet.tools import modelhub

#info = modelhub.download_dataset_for_model("numberplate_options")
#PATH_TO_DATASET = info["dataset_path"]

PATH_TO_DATASET = os.path.join(NOMEROFF_NET_DIR, "./data/dataset/OptionsDetector/numberplate_options_example")
#numberplate_options_example

In [4]:
PATH_TO_DATASET

'/mnt/data/var/www/nomeroff-net/./data/dataset/OptionsDetector/numberplate_options_example'

In [5]:
DATASET_NAME = "options"
VERSION = "2021_07_08_pytorch_lightning"

LOG_DIR = os.path.join(NOMEROFF_NET_DIR, "./data/logs/")
RESULT_PATH = os.path.join(NOMEROFF_NET_DIR, "./data/models/", 'numberplate_{}_{}.h5'.format(DATASET_NAME, VERSION))

In [6]:
from NomeroffNet import OptionsDetector

# definde your parameters
class MyNpClassificator(OptionsDetector):
    def __init__(self):
        OptionsDetector.__init__(self)
        
        # outputs 2
        self.class_region = [
            "xx-unknown", 
            "eu-ua-2015", 
            "eu-ua-2004", 
            "eu-ua-1995", 
            "eu", 
            "xx-transit", 
            "ru", 
            "kz", 
            "eu-ua-ordlo-dnr", 
            "eu-ua-ordlo-lnr", 
            "ge", 
            "by", 
            "su", 
            "kg",
            "am"
        ]
        
        # output 3
        self.count_lines = ["0", "1", "2", "3"]
        
        self.epochs           = 500
        self.batch_size       = 1
        self.gpus = 0
        
        self.height         = 64
        self.width         = 295


In [7]:
# initialize region detector.
npClassificator = MyNpClassificator()
npClassificator.prepare(PATH_TO_DATASET, verbose=1)

START PREPARING
DATA PREPARED


In [8]:
dm = npClassificator.dm

In [9]:
dm.prepare_data()

In [10]:
dm.setup()

In [11]:
# train
model = npClassificator.train()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

   | Name            | Type        | Params
-------------------------------------------------
0  | inp_conv        | Conv2d      | 896   
1  | conv1           | Conv2d      | 18.5 K
2  | conv2           | Conv2d      | 73.9 K
3  | conv3           | Conv2d      | 147 K 
4  | pool            | MaxPool2d   | 0     
5  | dropout_reg     | Dropout     | 0     
6  | fc1_reg         | Linear      | 2.1 M 
7  | fc2_reg         | Linear      | 131 K 
8  | batch_norm_reg  | BatchNorm1d | 1.0 K 
9  | fc3_reg         | Linear      | 3.9 K 
10 | dropout_line    | Dropout     | 0     
11 | fc1_line        | Linear      | 2.1 M 
12 | fc2_line        | Linear      | 131 K 
13 | batch_norm_line | BatchNorm1d | 1.0 K 
14 | fc3_line        | Linear      | 1.0 K 
-------------------------------------------------
4.7 M     Trainable params
0         Non-trainable params
4.7 M     Total params
18.823    Total estimated model params 

In [12]:
npClassificator.load(os.path.join(LOG_DIR, "./best.pb"))

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/var/www/nomeroff-net/./data/logs/./best.pb'

In [ ]:
npClassificator.test()

In [ ]:
npClassificator.save(RESULT_PATH)

In [ ]:
%matplotlib inline

In [ ]:
from NomeroffNet.Base.ImgGenerator import ImgGenerator
import torch
import json
import cv2
import matplotlib.pyplot as plt

def show_img(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show()

def small_acc(npClassificator, path_dir="../datasets/options/test", mode_torch="gpu", min_acc=0.75):
    imageGenerator = ImgGenerator(
        path_dir,
        npClassificator.WEIGHT,
        npClassificator.HEIGHT,
        npClassificator.BATCH_SIZE,
        [len(npClassificator.CLASS_REGION), len(npClassificator.CLASS_COUNT_LINE)])
    print("start imageGenerator build")
    imageGenerator.build_data()
    print("end imageGenerator build")
    gen = imageGenerator.path_generator()
    for i, (img_paths, inputs, labels) in enumerate(gen, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs = torch.from_numpy(inputs)
        if mode_torch == "gpu":
            inputs = inputs.cuda()
        outputs = npClassificator.MODEL(inputs)
        label_reg = torch.from_numpy(labels[0])
        label_cnt = torch.from_numpy(labels[1])
        if mode_torch == "gpu":
            label_reg = label_reg.cuda()
            label_cnt = label_cnt.cuda()
        out_idx_reg = torch.max(outputs[0], 1)[1].cpu().numpy()
        out_idx_line = torch.max(outputs[1], 1)[1].cpu().numpy()
        
        label_idx_reg = torch.max(label_reg, 1)[1].cpu().numpy()
        label_idx_line = torch.max(label_cnt, 1)[1].cpu().numpy()
        
        # check reg
        for o, l, p, tens in zip(out_idx_reg, label_idx_reg, img_paths, outputs[0]):
            if o != l:
                print("[WRONG REGION PREDICTED] pred", o, l, p)
                show_img(p)
            elif tens[l] < min_acc:
                print("[SMALL ACC REGION PREDICTED]  pred", o, l, tens[l], p)
                show_img(p)
        # check line
#         for o, l, p, tens in zip(out_idx_line, label_idx_line, img_paths, outputs[1]):
#             if o != l:
#                 print("[WRONG LINE PREDICTED]  pred", o, l, p)
#                 show_img(p)
#             elif tens[l] < min_acc:
#                 print("[SMALL ACC LINE PREDICTED]  pred", o, l, tens[l], p)
#                 show_img(p)

In [ ]:
small_acc(npClassificator, "../datasets/options/test")

In [ ]:
small_acc(npClassificator, "../datasets/options/train")

In [ ]:
small_acc(npClassificator, "../datasets/options/val")